In [2]:
import sys
print(sys.executable)
# Verify kernal path
import numpy as np
import pandas as pd
import os
import requests
from dotenv import load_dotenv
load_dotenv()

# time series dataframe
ts_prices = pd.read_parquet("../data/transformed/validated_prices_2022-08-02-2024-08-02.parquet") # reads in parquet-file in specified path and returns its data in a dataframe
ts_prices

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime,price_diff,volume_sma_5,volume_change,SMA_5,SMA_20,EMA_5,EMA_20,vol_5,daily_return
0,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-02 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
1,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-03 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
2,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-04 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
3,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-05 04:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
4,5.791877e+07,165.126767,164.263333,165.19,166.503333,163.40,507545.666667,2022-08-06 00:00:00,-0.160000,5.694134e+07,0.021549,165.318000,169.103000,165.296667,165.334762,0.071554,-0.000968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,3.515373e+07,218.131900,216.960000,218.24,219.300000,215.75,604680.000000,2024-07-29 04:00:00,0.093333,4.008290e+07,-0.042438,217.978000,225.414500,218.726415,221.994869,0.292077,0.000428
728,4.068162e+07,218.405900,219.190000,218.80,220.325000,216.12,584305.000000,2024-07-30 04:00:00,0.560000,3.812887e+07,0.157249,218.240000,224.705500,218.750943,221.690596,0.329983,0.002566
729,4.842297e+07,222.344100,221.440000,222.08,223.820000,220.63,668833.000000,2024-07-31 04:00:00,3.280000,3.984794e+07,0.190291,219.064000,224.431000,219.860629,221.727682,1.710839,0.014991
730,6.112524e+07,219.477300,224.370000,218.36,224.480000,217.02,876046.000000,2024-08-01 04:00:00,-3.720000,4.441905e+07,0.262319,219.125333,223.822000,219.360419,221.406950,1.670571,-0.016751


In [3]:
def get_cutoff_indicies(data, n_previous_days, step_size):
    stop_position = len(data)-1

    first_indx = 0
    last_indx = n_previous_days+1  # target index
    indicies = []

    while last_indx <= stop_position:
        indicies.append((first_indx, last_indx))

        first_indx += step_size
        last_indx += step_size
    
    return indicies

n_previous_days = 12  # make sure this constant across all uses
step_size = 1
indicies = get_cutoff_indicies(ts_prices, n_previous_days, step_size)
indicies[0:10]

[(0, 13),
 (1, 14),
 (2, 15),
 (3, 16),
 (4, 17),
 (5, 18),
 (6, 19),
 (7, 20),
 (8, 21),
 (9, 22)]

In [22]:
# (number_of_sequences, sequence_length, number_of_features)
# each sequence is an example, seq_len is number of days in each example, features is values per day
# number of cutoff indicies is each example
# for each cutoff indicies-pair get those days, create a matrix for each day add those days features as a row
# then add this matrix for this sequence to the total matrix as a single example
def transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size):
    indicies = get_cutoff_indicies(ts_prices, n_previous_days, step_size)
    indicies = indicies[:len(indicies)-1]  # exclude last pair
    num_seqs = len(indicies)
    print(f"Number of sequences/examples: {num_seqs}")

    features = []  
    targets = []

    for pair in indicies:
        start_indx = pair[0]  # start-index of day-row to end-index of day-row, all days in between this range is a sequence/example
        end_indx = pair[1]  # target indx of row

        sequence_df = ts_prices.iloc[start_indx: end_indx]  # get day-rows from start-indx to just before end-indx, each element is a day in current-sequence-day-example

        cur_seq_days_matrix = []  # each row is the values of features for each day in cur-sequence-example
        for _, day_row in sequence_df.iterrows():
            
            values = list(day_row.drop(labels="datetime").values)
            cur_seq_days_matrix.append(values)

        features.append(cur_seq_days_matrix)
        target_value = ts_prices.iloc[end_indx]["close_price"]   # target row is end-indx close-price
        targets.append(target_value)

        if pair == indicies[0]: # confirm last-val of seq is not same as the target next value
            print(f"Last val: {sequence_df.iloc[-1]["close_price"]}")
            print(f"Target val: {target_value}")

    features = np.array(features, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)
    return features, targets

print(indicies)
features, targets = transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size)
print(f"\nConfirm number of examples: {len(features) == len(targets)}")
print(f"Num-features-in-each-day: {len(ts_prices.columns)-1}")
print(f"N-previous-days: {n_previous_days}")
print(f"Step-size: {step_size}")
print(f'Features shape: {features.shape}')  # Should be (number_of_examples, n_previous_days, n_features)
print(f'Targets shape: {targets.shape}')    # Should be (number_of_examples,)
features
targets

[(0, 13), (1, 14), (2, 15), (3, 16), (4, 17), (5, 18), (6, 19), (7, 20), (8, 21), (9, 22), (10, 23), (11, 24), (12, 25), (13, 26), (14, 27), (15, 28), (16, 29), (17, 30), (18, 31), (19, 32), (20, 33), (21, 34), (22, 35), (23, 36), (24, 37), (25, 38), (26, 39), (27, 40), (28, 41), (29, 42), (30, 43), (31, 44), (32, 45), (33, 46), (34, 47), (35, 48), (36, 49), (37, 50), (38, 51), (39, 52), (40, 53), (41, 54), (42, 55), (43, 56), (44, 57), (45, 58), (46, 59), (47, 60), (48, 61), (49, 62), (50, 63), (51, 64), (52, 65), (53, 66), (54, 67), (55, 68), (56, 69), (57, 70), (58, 71), (59, 72), (60, 73), (61, 74), (62, 75), (63, 76), (64, 77), (65, 78), (66, 79), (67, 80), (68, 81), (69, 82), (70, 83), (71, 84), (72, 85), (73, 86), (74, 87), (75, 88), (76, 89), (77, 90), (78, 91), (79, 92), (80, 93), (81, 94), (82, 95), (83, 96), (84, 97), (85, 98), (86, 99), (87, 100), (88, 101), (89, 102), (90, 103), (91, 104), (92, 105), (93, 106), (94, 107), (95, 108), (96, 109), (97, 110), (98, 111), (99, 11

array([173.19   , 173.03   , 174.55   , 174.15   , 171.52   , 170.20334,
       168.88667, 167.57   , 167.23   , 167.53   , 170.03   , 163.62   ,
       162.87334, 162.12666, 161.38   , 158.91   , 157.22   , 157.96   ,
       155.81   , 155.49   , 155.17   , 154.85   , 154.53   , 155.96   ,
       154.46   , 157.37   , 159.39   , 161.41   , 163.43   , 153.84   ,
       155.31   , 152.37   , 150.7    , 151.96   , 153.22   , 154.48   ,
       156.9    , 153.72   , 152.74   , 150.43   , 150.54333, 150.65666,
       150.77   , 151.76   , 149.84   , 142.48   , 138.2    , 139.61667,
       141.03334, 142.45   , 146.1    , 146.4    , 145.43   , 140.09   ,
       140.2    , 140.31   , 140.42   , 138.98   , 138.34   , 142.99   ,
       138.38   , 139.72333, 141.06667, 142.41   , 143.75   , 143.86   ,
       143.39   , 147.27   , 147.99667, 148.72333, 149.45   , 152.34   ,
       149.35   , 144.8    , 155.74   , 154.94   , 154.14   , 153.34   ,
       150.65   , 145.03   , 138.88   , 138.38   , 

In [15]:
tabular_data = pd.DataFrame(features)
tabular_data["target_close_price_next_day"] = pd.DataFrame(targets)
tabular_data
# each column is a dayx-featurey

,0,1,2,3,4,5,6,7,8,9,...,212,213,214,215,216,217,218,219,220,target_close_price_next_day
0,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659398e+18,0.000000,56941340.0,...,0.363333,63496620.0,-0.073343,171.024002,169.102997,170.947968,167.599106,2.005173,0.002107,173.190002
1,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659485e+18,0.000000,56941340.0,...,0.363333,60280852.0,-0.079148,171.813995,169.102997,171.695312,168.131577,1.902056,0.002102,173.029999
2,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659571e+18,0.000000,56941340.0,...,-0.160000,60127840.0,0.042250,172.722000,169.102997,172.140213,168.598099,0.441202,-0.000924,174.550003
3,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659672e+18,0.000000,56941340.0,...,1.520000,62428372.0,0.410894,173.212006,169.102997,172.943466,169.164948,0.795744,0.008785,174.149994
4,57918768.0,165.126770,164.263336,165.190002,166.503326,163.399994,507545.65625,1.659744e+18,-0.160000,56941340.0,...,-0.400000,62208356.0,-0.216891,173.549332,169.102997,173.345642,169.639709,0.755515,-0.002292,171.520004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,38468544.0,234.299194,235.000000,234.820007,236.270004,232.330002,623906.00000,1.721102e+18,0.420000,52547780.0,...,0.093333,44989644.0,-0.040710,218.037994,225.936493,218.969620,222.390121,0.377636,0.000428,218.240005
715,55878904.0,228.839600,229.449997,228.880005,231.459900,226.639999,811024.00000,1.721189e+18,-5.940000,53399272.0,...,0.093333,40082904.0,-0.042438,217.977997,225.414505,218.726410,221.994873,0.292077,0.000428,218.800003
716,64346032.0,225.181000,230.279999,224.179993,230.440002,222.270004,844538.00000,1.721275e+18,-4.700000,55351372.0,...,0.560000,38128876.0,0.157249,218.240005,224.705505,218.750946,221.690598,0.329983,0.002566,222.080002
717,48020040.0,224.735596,224.820007,224.309998,226.800003,223.274994,612739.00000,1.721362e+18,0.130000,53445452.0,...,3.280000,39847940.0,0.190291,219.063995,224.431000,219.860626,221.727676,1.710839,0.014991,218.360001


In [16]:
# save features/targets tabular data
tabular_data.to_parquet("../data/transformed/tabular_prices_2022-08-02-2024-08-02.parquet")

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/lib/python3.12/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



model = GradientBoostingRegressor(n_estimators=100, random_state=42)
model.fit(features, targets)

predictions = model.predict(features)
mse = mean_squared_error(targets, predictions)
print(f'Mean Squared Error: {mse}')
print(len(features[0]))

Mean Squared Error: 0.9815958805094114
221


In [65]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(features, targets)

# Make predictions and calculate MSE
y_pred = rf_model.predict(features)
mse = mean_squared_error(targets, y_pred)
print(f'Random Forest MSE: {mse}')

Random Forest MSE: 0.7365136091476493
